DATARESEARCH2: EN este documento nos centraremos en obtener tablas importantes para nuestra seleccion de paises, la inflacion, o deflactor del PIB, es un factor precursor de politicas negativas para el bienestar del individuo,
tambien se puede relacionar con el PIB PER CAPITA, el SMI y el Sueldo bruto Anual Medio.
Vamos a relacionar datos economicos con otros factores importantes a la hora de agrupar paises como la zona geografica y similitudes culturales.

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import WebDriverException
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
import requests
import random 
import pandas as pd
import numpy as np
import sqlite3
import time
import os

In [ ]:

driver = webdriver.Chrome() 
url = "https://datos.bancomundial.org/indicador/NY.GDP.DEFL.KD.ZG?contextual=region&end=2023&locations=EU-DK&most_recent_value_desc=false&start=1961&view=chart"
driver.get("url") 
mi_programa --user-data-dir=Documentos/datos_sesion2
# Find all the "item" divs containing the data
item_divs = driver.find_elements(By.CLASS_NAME, "item") 

# Create an empty list to store the extracted data
inflacion = []

# Iterate through each "item" div
for item in item_divs:
    # Extract country name
    country_name = item.find_element(By.CLASS_NAME, "country-name").text 
    
    # Extract year
    year = item.find_elements(By.TAG_NAME, "div")[1].text 
    
    # Extract value
    value = item.find_elements(By.TAG_NAME, "div")[2].text 
    
    # Append the extracted data to the list
    inflacion.append({"PaisEuro": country_name, "Año": year, "Inflacion": value})


df_inf= pd.DataFrame(inflacion)

# Print the DataFrame
print(df_inf)

# Close the WebDriver
driver.quit()